<a href="https://colab.research.google.com/github/hruthiksiva/DataScience-Competitions/blob/main/enigma_datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extract csv

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
! kaggle competitions download -c domain-network-security


  0% 0.00/746k [00:00<?, ?B/s]
100% 746k/746k [00:00<00:00, 50.1MB/s]
  0% 0.00/238k [00:00<?, ?B/s]
100% 238k/238k [00:00<00:00, 77.8MB/s]
  0% 0.00/693k [00:00<?, ?B/s]
100% 693k/693k [00:00<00:00, 136MB/s]


In [8]:
!unzip \*.zip  && rm *.zip

Archive:  train.csv.zip
  inflating: train.csv               


## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Collection/ Alteration

In [106]:
df=pd.read_csv('train.csv')

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45870 entries, 0 to 45869
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    45870 non-null  int64  
 1   Source Port           36683 non-null  float64
 2   Destination Port      36693 non-null  float64
 3   NAT Source Port       36756 non-null  float64
 4   NAT Destination Port  36674 non-null  float64
 5   Bytes                 36607 non-null  float64
 6   Bytes Sent            36764 non-null  float64
 7   Bytes Received        36693 non-null  float64
 8   Packets               36818 non-null  float64
 9   Elapsed Time (sec)    36584 non-null  float64
 10  pkts_sent             36667 non-null  float64
 11  pkts_received         36711 non-null  float64
 12  Action                45870 non-null  object 
dtypes: float64(11), int64(1), object(1)
memory usage: 4.5+ MB


In [75]:
df.isnull().sum()

ID                         0
Source Port             9187
Destination Port        9177
NAT Source Port         9114
NAT Destination Port    9196
Bytes                   9263
Bytes Sent              9106
Bytes Received          9177
Packets                 9052
Elapsed Time (sec)      9286
pkts_sent               9203
pkts_received           9159
Action                     0
dtype: int64

### Encoding Data

In [108]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
df['Action']= lb.fit_transform(df['Action'])

In [109]:
df['Action'].describe()

count    45870.000000
mean         0.623305
std          0.794887
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          3.000000
Name: Action, dtype: float64

### Float to Int

In [122]:
list =['Source Port','Destination Port','NAT Source Port','NAT Destination Port','Bytes',
       'Bytes Sent','Bytes Received','Packets','Elapsed Time (sec)','pkts_sent','pkts_received']

for i in list:
  df[str(i)] = df[str(i)].fillna(-1)
  df[str(i)]=df[str(i)].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45870 entries, 0 to 45869
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   ID                    45870 non-null  int64
 1   Source Port           45870 non-null  int64
 2   Destination Port      45870 non-null  int64
 3   NAT Source Port       45870 non-null  int64
 4   NAT Destination Port  45870 non-null  int64
 5   Bytes                 45870 non-null  int64
 6   Bytes Sent            45870 non-null  int64
 7   Bytes Received        45870 non-null  int64
 8   Packets               45870 non-null  int64
 9   Elapsed Time (sec)    45870 non-null  int64
 10  pkts_sent             45870 non-null  int64
 11  pkts_received         45870 non-null  int64
 12  Action                45870 non-null  int64
dtypes: int64(13)
memory usage: 4.5 MB


### Separate Label vs Feature

In [123]:
y=df.iloc[:,[-1]]
X = df.drop(['ID','Action'],axis=1)

## Test Train Split

In [124]:
from sklearn.model_selection import train_test_split

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

# Data Visualization

In [115]:
print(df['ID'].corr(df['Source Port']))
print(df['pkts_received'].corr(df['Action']))
print(df['ID'].corr(df['Action']))

0.004004893499190068
-0.018264771302748367
-6.193059199356068e-05


# Model Decision

### KNN Algorithm

In [116]:
from sklearn.neighbors import KNeighborsClassifier

In [117]:
model = KNeighborsClassifier(n_neighbors=4)
model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

### XGBooster

In [121]:
import xgboost as xgb
xgb.XGBClassifier()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [126]:
model=xgb.XGBClassifier(missing=-1)
model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=-1, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

### Random forest

In [102]:
from sklearn.ensemble import RandomForestClassifier

In [103]:
model=RandomForestClassifier()
model .fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Prediction

In [127]:
y_pred=model.predict(X_test)

In [128]:
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test, y_pred))
#xgbooster - 0.985
#random forest - 0.984
#knn - 0.95

Accuracy: 0.985829518203619


In [ ]:
# calculates precision for 1:100 dataset with 90 tp and 30 fp
from sklearn.metrics import precision_score
# define actual
act_pos = [1 for _ in range(100)]
act_neg = [0 for _ in range(10000)]
y_true = act_pos + act_neg
# define predictions
pred_pos = [0 for _ in range(10)] + [1 for _ in range(90)]
pred_neg = [1 for _ in range(30)] + [0 for _ in range(9970)]
y_pred = pred_pos + pred_neg
# calculate prediction
precision = precision_score(y_true, y_pred, average='binary')
print('Precision: %.3f' % precision)

# Data Extraction

In [ ]:
tdf=pd.read_csv('test.csv')

In [ ]:
le=LabelEncoder()
df['Action']= le.fit_transform(df['Action'])

In [ ]:
list =['Source Port','Destination Port','NAT Source Port','NAT Destination Port','Bytes',
       'Bytes Sent','Bytes Received','Packets','Elapsed Time (sec)','pkts_sent','pkts_received']

for i in list:
  tdf[str(i)] = tdf[str(i)].fillna(-1)
  tdf[str(i)]=tdf[str(i)].astype(int)
tdf.info()

In [ ]:
X_=tdf.drop(y.columns,axis = 1)

In [ ]:
ypred_=model.predict(X_)

In [ ]:
ypred_ = le.inverse_transform(ypred_)

In [ ]:
#dont forget to decode labels back to open closed etc
sub=pd.DataFrame({"id":tdf["id"],"target":np.array(ypred_).reshape(-1)})
sub.to_csv("submission1.csv",index=False)